In [1]:
import numpy as np
from itertools import product
import math
import random
import copy

According to Hinton's "A Practical Guide to Training Restricted Boltzmann Machines"

Initialize J matrix to zero mean and 0.01 sd. The diagonal will be 0.

Initizlize bias b_i  of visible unit i to log[pi/(1−pi)]

In [2]:
class IsingModel:
    def __init__(self, data, lr=0.1):
        self.lr = lr
        
        # Possible values the spins can take
        self.spin_values = [-1,1]
        
        # Create a ndarray from the data
        self.data_activations_matrix = np.array(data)
        # Calculate the shape of the matrix and save useful variables
        self.num_spins = self.data_activations_matrix.shape[1]
        self.num_samples_data = self.data_activations_matrix.shape[0]
        
        # Calculate the true data expectations
        # First calculate the correlation
        self.data_correlation = self.calculate_corr(self.data_activations_matrix)
        # Then, the mean
        self.data_mean = np.mean(self.data_activations_matrix, axis=0)
        
        
        # Calculate the initial J matrix. Fill only the upper diagonal
        # Filling with Normal(0, 0.01)
        np.random.seed(3)
        self.J = np.random.normal(loc=0.0, scale=.01, size=self.data_correlation.shape)
        lind = np.tril_indices_from(self.J)
        self.J[lind]=0
                
        # Calculate the initial H vector, each entry proportional to its probability (according to Hinton)
        #self.H = np.sum(self.data_activations_matrix, axis=0)/ float(self.num_samples_data)
        # Try with random H
        self.H = np.random.normal(loc=0.0, scale=.01, size=self.num_spins)
        
    # Calculate the correlation of a matrix
    def calculate_corr(self, activations_matrix):
        num_spins = activations_matrix.shape[1]
        num_samples_data = activations_matrix.shape[0]
        
        corr_matrix = np.zeros((self.num_spins, self.num_spins))
        for sigma in activations_matrix:
            # Calculate \sigma_i * \sigma_j
            for i in range(0, self.num_spins):
                for j in range(i+1, self.num_spins):
                    corr_matrix[i][j] += sigma[i]*sigma[j]
        corr_matrix = corr_matrix / float(self.num_samples_data)
        
        return corr_matrix
    
    # Calcualte the energy of the model's parameters given a sigma state
    def calculate_energy(self, sigma):
        energy = 0
        for i in range(0, self.num_spins):
            for j in range(i+1, self.num_spins):
                energy += sigma[i]*sigma[j]*self.J[i][j]

        for i in range(0, self.num_spins):
            energy += sigma[i]*self.H[i]

        return - energy
                
    
    # Flip a randomly selected spin
    def flip_spin(self, _sigma):
        sigma = copy.deepcopy(_sigma)
        spin_to_flip = random.choice(range(0, self.num_spins))

        spin = sigma[spin_to_flip]
        if spin == self.spin_values[0]:
            sigma[spin_to_flip] = self.spin_values[1]
        else:
            sigma[spin_to_flip] = self.spin_values[0]
        return sigma
        
        
    
    # Train the exact model
    def train(self, max_epochs = 500):
        
        totalParamVariation = math.inf
        stopCondition = 0.000005
        
        epoch = 1
        
        while totalParamVariation > stopCondition and epoch < max_epochs:
            # Calculate P(sigma) for every possible combination in the model

            prob_dict = {}
            for sigma in product(self.spin_values, repeat=self.num_spins):
                prob_dict[sigma] = math.exp(-self.calculate_energy(sigma))

            # Calculate the partition function
            Z = sum(prob_dict.values())

            # Normalize dividing by the partition function
            for sigma in prob_dict.keys():
                prob_dict[sigma] = prob_dict[sigma]/Z

            model_correlation = np.zeros(self.J.shape)
            model_mean = np.zeros(self.H.shape)
            for sigma in product(self.spin_values, repeat=self.num_spins):
                
                # Calculate the expectation of the correlations
                for i in range(0, self.num_spins):
                    for j in range(i+1, self.num_spins):
                        model_correlation[i][j] += sigma[i]*sigma[j]*prob_dict[sigma]

                # Calculate the expectation of the averages
                for i in range(0, self.num_spins):
                    model_mean[i] += sigma[i]*prob_dict[sigma]


            # Calculate the step size for every J_{ij}
            stepJ = self.lr * (self.data_correlation - model_correlation)
            # Take the step
            oldJ = copy.deepcopy(self.J)
            self.J = self.J + stepJ

            # Calculate the step size for every H_{i}
            stepH = self.lr * (self.data_mean - model_mean)
            # Calculate the variation of J for the termination condition
            # Take the step
            oldH = copy.deepcopy(self.H)
            self.H = self.H + stepH
            
            diffH = np.sum(np.absolute(self.H - oldH))
            diffJ = np.sum(np.sum(np.absolute(self.J - oldJ)))
            totalParamVariation = diffJ + diffH
            
            if epoch%100 == 0:
                print('Epoch', epoch, 'TotalParamVariation', round(totalParamVariation, 8))
                print(model_correlation, 'Corr Model')
                print(self.data_correlation, 'Corr Data')

                print(model_mean, 'Average Model')
                print(self.data_mean, 'Average Data')
                print()
            
            epoch += 1

        print(self.H, 'H')
        print(self.J, 'J')
        
    def random_walk(self, max_steps):
        # Generate a random initial spin configuration
        model_activations_list = []
        sigma_0 = random.choices([-1,1], k=self.num_spins)
        
        # Calculate the energy at this time
        energy_0 = self.calculate_energy(sigma_0)
        
        for i in range(0, max_steps):
            
            # Flip a spin and calculate the new energy
            sigma_t = self.flip_spin(sigma_0)
            energy_t = self.calculate_energy(sigma_t)
            delta_energy = energy_t - energy_0

            if delta_energy<=0: # If delta_energy<0, accept the spin flip, energetically favourable
                model_activations_list.append(sigma_t)
            

            else: # Check if transition probability p < uniform r
                p = math.exp(-delta_energy)
                r = random.random()

                if r<=p: # Accept spin flip
                    model_activations_list.append(sigma_t)
                else: # Refuse spin flip
                    model_activations_list.append(sigma_0)
            
            sigma_0 = sigma_t
            energy_0 = energy_t
        
        return model_activations_list
    
    def train_metropolis(self, max_epochs = 300, random_walk_len = 100):
        random.seed(3)
        totalParamVariation = math.inf
        stopCondition = 0.0005
        
        epoch = 1
        
        while totalParamVariation > stopCondition and epoch < max_epochs:
            model_activations_list = self.random_walk(random_walk_len)  
            model_activations_matrix = np.array(model_activations_list)
            # Calculate the data expectations
            model_correlation = self.calculate_corr(model_activations_matrix)
            model_mean = np.mean(model_activations_matrix, axis=0)
            
            # Calculate the step size for every J_{ij}
            stepJ = self.lr * (self.data_correlation - model_correlation)
            
            oldJ = copy.deepcopy(self.J)
            # Take the step
            self.J = self.J + stepJ

            # Calculate the step size for every H_{i}
            stepH = self.lr * (self.data_mean - model_mean)
            
            oldH = copy.deepcopy(self.H)
            # Take the step
            self.H = self.H + stepH
            
            # Calculate the variation of J and H for the termination condition
            diffH = np.sum(np.absolute(self.H - oldH))
            diffJ = np.sum(np.sum(np.absolute(self.J - oldJ)))
            totalParamVariation = diffJ + diffH
            
            if epoch%10 == 0:
                print('Epoch', epoch, 'TotalParamVariation', round(totalParamVariation, 8))
                print(model_correlation, 'Corr Model')
                print(self.data_correlation, 'Corr Data')

                print(model_mean, 'Average Model')
                print(self.data_mean, 'Average Data')
                print()
                                    
            epoch +=1

        print(self.H, 'H')
        print(self.J, 'J')
        

In [3]:
activations_list = [[1,-1,1,-1], [1,-1,1,-1], [-1,1,-1,-1], [-1,1,1,1], [1,1,1,-1], [-1,-1,1,1], [1,-1,-1,1], [1,-1,1,1]]

In [4]:
ising = IsingModel(activations_list, lr=0.005)

In [15]:
ising.train_metropolis(max_epochs = 15000, random_walk_len = 400)


Epoch 10 TotalParamVariation 0.08225
[[ 0.   2.5  3.5 -3. ]
 [ 0.   0.  -2.  -5. ]
 [ 0.   0.   0.   0.5]
 [ 0.   0.   0.   0. ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.33 -0.17  0.26 -0.05] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 20 TotalParamVariation 0.066375
[[ 0.    2.5  -2.75  1.  ]
 [ 0.    0.    3.75  0.5 ]
 [ 0.    0.    0.    0.75]
 [ 0.    0.    0.    0.  ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.095 -0.205  0.35  -0.175] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 30 TotalParamVariation 0.0927
[[ 0.    6.   -2.25 -0.25]
 [ 0.    0.    4.25 -0.75]
 [ 0.    0.    0.    4.  ]
 [ 0.    0.    0.    0.  ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.22  -0.25   0.045 -0

Epoch 250 TotalParamVariation 0.07925
[[ 0.   -5.    2.75  2.75]
 [ 0.    0.    0.75  4.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.245 -0.265  0.22  -0.05 ] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 260 TotalParamVariation 0.1307
[[ 0.   -2.25  8.5   6.25]
 [ 0.    0.   -4.25  1.  ]
 [ 0.    0.    0.    4.25]
 [ 0.    0.    0.    0.  ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.265 -0.23   0.475  0.08 ] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 270 TotalParamVariation 0.058275
[[ 0.    1.75 -0.5  -1.  ]
 [ 0.    0.   -3.25 -0.25]
 [ 0.    0.    0.    4.5 ]
 [ 0.    0.    0.    0.  ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[

Epoch 480 TotalParamVariation 0.066775
[[ 0.    5.25 -0.25 -3.75]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.   -2.5 ]
 [ 0.    0.    0.    0.  ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.055 -0.19   0.17  -0.02 ] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 490 TotalParamVariation 0.09475
[[ 0.    1.25 -2.75 -2.  ]
 [ 0.    0.   -5.5  -1.75]
 [ 0.    0.    0.    5.25]
 [ 0.    0.    0.    0.  ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.07  -0.225  0.335  0.08 ] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 500 TotalParamVariation 0.0777
[[ 0.   -5.75 -1.   -1.25]
 [ 0.    0.   -1.75 -0.5 ]
 [ 0.    0.    0.   -5.75]
 [ 0.    0.    0.    0.  ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[

Epoch 760 TotalParamVariation 0.035975
[[ 0.    1.5  -0.5   1.75]
 [ 0.    0.    1.5  -0.25]
 [ 0.    0.    0.   -0.25]
 [ 0.    0.    0.    0.  ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.195 -0.265  0.165 -0.04 ] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 770 TotalParamVariation 0.076275
[[ 0.    0.5  -5.25 -4.  ]
 [ 0.    0.   -0.75  0.  ]
 [ 0.    0.    0.    3.75]
 [ 0.    0.    0.    0.  ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.19  -0.13   0.235 -0.06 ] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 780 TotalParamVariation 0.081375
[[ 0.    0.   -2.75 -1.5 ]
 [ 0.    0.    0.75  7.  ]
 [ 0.    0.    0.   -2.75]
 [ 0.    0.    0.    0.  ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Dat

Epoch 1000 TotalParamVariation 0.103875
[[ 0.    8.25 -5.75 -0.75]
 [ 0.    0.   -3.   -1.  ]
 [ 0.    0.    0.    1.5 ]
 [ 0.    0.    0.    0.  ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.075 -0.19   0.31  -0.1  ] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 1010 TotalParamVariation 0.0832
[[ 0.    2.25 -3.75  2.5 ]
 [ 0.    0.    1.   -2.25]
 [ 0.    0.    0.    3.25]
 [ 0.    0.    0.    0.  ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.185 -0.17   0.11   0.105] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 1020 TotalParamVariation 0.09925
[[ 0.   -6.25  5.5   4.25]
 [ 0.    0.   -3.75 -0.5 ]
 [ 0.    0.    0.   -0.25]
 [ 0.    0.    0.    0.  ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Dat

...

Epoch 13840 TotalParamVariation 0.055375
[[ 0.    1.75  1.75 -3.75]
 [ 0.    0.   -2.    1.  ]
 [ 0.    0.    0.    0.5 ]
 [ 0.    0.    0.    0.  ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.21  -0.225  0.255  0.015] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 13850 TotalParamVariation 0.0749
[[ 0.   -1.25  0.75  5.  ]
 [ 0.    0.    2.   -5.25]
 [ 0.    0.    0.    0.75]
 [ 0.    0.    0.    0.  ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.265 -0.24   0.17   0.125] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 13860 TotalParamVariation 0.09655
[[ 0.   -6.5   3.25  0.25]
 [ 0.    0.   -7.25  2.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr 

Epoch 14100 TotalParamVariation 0.060575
[[ 0.    2.75 -0.25 -0.25]
 [ 0.    0.    4.    2.5 ]
 [ 0.    0.    0.    1.  ]
 [ 0.    0.    0.    0.  ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.18  -0.135  0.345  0.025] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 14110 TotalParamVariation 0.103625
[[ 0.    0.5   4.   -1.75]
 [ 0.    0.   -2.    6.25]
 [ 0.    0.    0.   -5.75]
 [ 0.    0.    0.    0.  ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.4   -0.25   0.22  -0.045] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 14120 TotalParamVariation 0.081625
[[ 0.    5.75 -0.25  3.25]
 [ 0.    0.    0.5   3.  ]
 [ 0.    0.    0.    1.5 ]
 [ 0.    0.    0.    0.  ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Co

Epoch 14370 TotalParamVariation 0.0628
[[ 0.   2.5 -1.  -3. ]
 [ 0.   0.  -1.  -0.5]
 [ 0.   0.   0.  -4. ]
 [ 0.   0.   0.   0. ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.175 -0.115  0.325 -0.175] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 14380 TotalParamVariation 0.060775
[[ 0.    0.75  5.75 -1.75]
 [ 0.    0.    1.    1.  ]
 [ 0.    0.    0.    1.  ]
 [ 0.    0.    0.    0.  ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.245 -0.1    0.3    0.05 ] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 14390 TotalParamVariation 0.085725
[[ 0.   -7.    0.25 -1.  ]
 [ 0.    0.    3.75 -0.5 ]
 [ 0.    0.    0.    5.25]
 [ 0.    0.    0.    0.  ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.175 -0

Epoch 14600 TotalParamVariation 0.066175
[[ 0.   -7.   -0.25 -3.  ]
 [ 0.    0.   -1.25 -2.  ]
 [ 0.    0.    0.    0.25]
 [ 0.    0.    0.    0.  ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.295 -0.265  0.2   -0.125] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 14610 TotalParamVariation 0.040275
[[ 0.   -0.25  1.   -1.  ]
 [ 0.    0.   -4.25  1.75]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.25  -0.165  0.35  -0.07 ] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 14620 TotalParamVariation 0.08205
[[ 0.    1.5  -5.25 -3.25]
 [ 0.    0.   -0.75 -2.75]
 [ 0.    0.    0.   -2.5 ]
 [ 0.    0.    0.    0.  ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Cor

Epoch 14840 TotalParamVariation 0.127375
[[ 0.    2.75  3.5   2.5 ]
 [ 0.    0.   -9.75  3.75]
 [ 0.    0.    0.   -2.5 ]
 [ 0.    0.    0.    0.  ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.225 -0.35   0.405 -0.005] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 14850 TotalParamVariation 0.083025
[[ 0.   -1.    0.    0.75]
 [ 0.    0.   -8.5  -3.25]
 [ 0.    0.    0.   -3.25]
 [ 0.    0.    0.    0.  ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.275 -0.345  0.275  0.01 ] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 14860 TotalParamVariation 0.108575
[[ 0.   -6.   -7.75 -5.  ]
 [ 0.    0.    1.25  1.  ]
 [ 0.    0.    0.    0.25]
 [ 0.    0.    0.    0.  ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Co

In [5]:
ising.train(max_epochs = 10000)

<class 'tuple'>


KeyError: (-1, -1, -1, 1)

#### Only H

In [19]:
class SimpleIsingModel:
    def __init__(self, data, lr=0.0005):
        self.lr = lr
        
        # Possible values the spins can take
        self.spin_values = [-1,1]
        
        # Create a ndarray from the data
        self.data_activations_matrix = np.array(data)
        # Calculate the shape of the matrix and save useful variables
        self.num_spins = self.data_activations_matrix.shape[1]
        self.num_samples_data = self.data_activations_matrix.shape[0]
        
        # Calculate the true data expectations
        # The mean
        self.data_mean = np.mean(self.data_activations_matrix, axis=0)
        
                
        # Calculate the initial H vector, each entry proportional to its probability (according to Hinton)
        #self.H = np.sum(self.data_activations_matrix, axis=0)/ float(self.num_samples_data)
        # Try with random H
        self.H = np.random.normal(loc=0.0, scale=.01, size=self.num_spins)
    
    # Calcualte the energy of the model's parameters given a sigma state
    def calculate_energy(self, sigma):
        energy = 0
        for i in range(0, self.num_spins):
            energy += sigma[i]*self.H[i]

        return - energy
    
    # Flip a randomly selected spin
    def flip_spin(self, _sigma):
        sigma = copy.deepcopy(_sigma)
        spin_to_flip = random.choice(range(0, self.num_spins))

        spin = sigma[spin_to_flip]
        if spin == self.spin_values[0]:
            sigma[spin_to_flip] = self.spin_values[1]
        else:
            sigma[spin_to_flip] = self.spin_values[0]
        return sigma
        
        
    # Train the exact model
    def train(self, max_epochs = 500):
        
        totalParamVariation = math.inf
        stopCondition = 0.0000005
        
        epoch = 1
        
        while totalParamVariation > stopCondition and epoch < max_epochs:
            # Calculate P(sigma) for every possible combination in the model

            prob_dict = {}
            for sigma in product(self.spin_values, repeat=self.num_spins):
                prob_dict[sigma] = math.exp(-self.calculate_energy(sigma))

            # Calculate the partition function
            Z = sum(prob_dict.values())

            # Normalize dividing by the partition function
            for sigma in prob_dict.keys():
                prob_dict[sigma] = prob_dict[sigma]/Z

            model_mean = np.zeros(self.H.shape)
            for sigma in product(self.spin_values, repeat=self.num_spins):
                # Calculate the expectation of the averages
                for i in range(0, self.num_spins):
                    model_mean[i] += sigma[i]*prob_dict[sigma]


            # Calculate the step size for every H_{i}
            stepH = self.lr * (self.data_mean - model_mean)
            
            oldH = copy.deepcopy(self.H)
            # Take the step
            self.H = self.H + stepH
            
            # Calculate the variation of J for the termination condition
            totalParamVariation = np.sum(np.absolute(self.H - oldH))

            if epoch%100 == 0:
                print('Epoch', epoch, 'TotalParamVariation', round(totalParamVariation, 8))
                print(model_mean, 'Average Model')
                print(self.data_mean, 'Average Data')
                print(self.data_mean - model_mean, 'Diff')
                print()
            
            epoch += 1

        print(self.data_mean, 'Spins average')
        print(self.H, 'H')
        
    def random_walk(self, max_steps):
        # Generate a random initial spin configuration
        model_activations_list = []
        sigma_0 = random.choices([-1,1], k=self.num_spins)
        burn_in_len = 100
        
        # Calculate the energy at this time
        energy_0 = self.calculate_energy(sigma_0)
        
        for i in range(0, max_steps):
            
            # Flip a spin and calculate the new energy
            sigma_t = self.flip_spin(sigma_0)
            energy_t = self.calculate_energy(sigma_t)
            delta_energy = energy_t - energy_0

            # If delta_energy<0, accept the spin flip, energetically favourable
            if delta_energy>0:  # Ohterwise, check if transition probability p < uniform r

                p = math.exp(-delta_energy)
                r = random.random()

                if r>p: # If transition probability is lower than r, refuse the update, otherwise accept
                    sigma_t = copy.deepcopy(sigma_0)
                    energy_t = energy_0
            
            # Save the state if the burn-in phase has passed
            if i > burn_in_len:
                model_activations_list.append(sigma_t)

            sigma_0 = copy.deepcopy(sigma_t)
            energy_0 = energy_t
        
        return model_activations_list
    
    def train_metropolis(self, max_epochs = 300, random_walk_len = 100):
        random.seed(3)
        totalParamVariation = math.inf
        stopCondition = 0.0000005
        
        epoch = 1
        
        print('Epoch 0')
        print(self.H)
        
        while totalParamVariation > stopCondition and epoch < max_epochs:
            model_activations_list = self.random_walk(random_walk_len)   
            model_activations_matrix = np.array(model_activations_list)

            # Calculate the data expectations
            model_mean = np.mean(model_activations_matrix, axis=0)
            
            # Calculate the step size for every H_{i}
            stepH = self.lr * (self.data_mean - model_mean)
            # Take the step
            oldH = copy.deepcopy(self.H)
            self.H = self.H + stepH
            
            # Calculate the variation of H for the stop condition
            totalParamVariation = np.sum(np.absolute(self.H - oldH))
            
            if epoch%10 == 0:
                print('Epoch', epoch, 'TotalParamVariation', round(totalParamVariation, 8))
                print(model_mean, 'Average Model')
                print(self.data_mean, 'Average Data')
                print(self.data_mean - model_mean, 'Diff')
                print()

            
            epoch +=1
        
        print(self.data_mean, 'Spins average')
        print(self.H, 'H')


In [23]:
activations_list = [[1,-1,1,-1], [1,-1,1,-1], [-1,1,-1,-1], [-1,1,1,1], [1,1,1,-1], [-1,-1,1,1], [1,-1,-1,1], [1,-1,1,1]]
s_ising = SimpleIsingModel(activations_list, lr=0.0005)


In [22]:
s_ising.train_metropolis( max_epochs = 15000, random_walk_len = 400)

Epoch 0
[ 0.00062221 -0.0100401  -0.00183667 -0.00556573]
Epoch 10 TotalParamVariation 0.00033612
[ 0.08361204 -0.12374582  0.15719064 -0.0367893 ] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.16638796 -0.12625418  0.34280936  0.0367893 ] Diff

Epoch 20 TotalParamVariation 0.00057692
[ 0.090301    0.07692308 -0.09698997  0.07023411] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.159699   -0.32692308  0.59698997 -0.07023411] Diff

Epoch 30 TotalParamVariation 0.00048662
[-0.06354515 -0.04347826  0.08361204  0.0367893 ] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.31354515 -0.20652174  0.41638796 -0.0367893 ] Diff

Epoch 40 TotalParamVariation 0.0005
[-0.08361204 -0.07692308  0.07023411  0.06354515] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.33361204 -0.17307692  0.42976589 -0.06354515] Diff

Epoch 50 TotalParamVariation 0.00049666
[-0.07023411 -0.10367893 -0.01003344  0.01672241] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.320

[ 0.0367893  -0.01003344  0.25752508  0.090301  ] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.2132107  -0.23996656  0.24247492 -0.090301  ] Diff

Epoch 450 TotalParamVariation 0.00060702
[-0.04347826  0.21070234  0.1638796   0.12374582] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.29347826 -0.46070234  0.3361204  -0.12374582] Diff

Epoch 460 TotalParamVariation 0.00057023
[ 0.02341137  0.00334448  0.01672241 -0.17725753] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.22658863 -0.25334448  0.48327759  0.17725753] Diff

Epoch 470 TotalParamVariation 0.00060368
[ 0.0367893  -0.03010033 -0.090301   -0.18394649] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.2132107  -0.21989967  0.590301    0.18394649] Diff

Epoch 480 TotalParamVariation 0.00055017
[-0.01672241  0.04347826  0.05685619 -0.09698997] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.26672241 -0.29347826  0.44314381  0.09698997] Diff

Epoch 490 TotalParamVariation 0.00041639


Epoch 860 TotalParamVariation 0.0003194
[ 0.15050167 -0.09698997  0.19063545  0.07692308] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.09949833 -0.15301003  0.30936455 -0.07692308] Diff

Epoch 870 TotalParamVariation 0.00026589
[ 0.08361204 -0.07023411  0.38461538  0.07023411] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.16638796 -0.17976589  0.11538462 -0.07023411] Diff

Epoch 880 TotalParamVariation 0.00028261
[ 0.17725753 -0.13043478  0.33110368  0.20401338] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.07274247 -0.11956522  0.16889632 -0.20401338] Diff

Epoch 890 TotalParamVariation 0.00041304
[ 0.09698997 -0.0367893   0.17725753 -0.13712375] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.15301003 -0.2132107   0.32274247  0.13712375] Diff

Epoch 900 TotalParamVariation 0.00041639
[-0.01672241 -0.06354515  0.17725753  0.05685619] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.26672241 -0.18645485  0.32274247 -0.05685619] Diff

E

Epoch 1270 TotalParamVariation 0.00024247
[ 0.08361204 -0.15719064  0.35785953  0.08361204] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.16638796 -0.09280936  0.14214047 -0.08361204] Diff

Epoch 1280 TotalParamVariation 0.00048997
[-0.17056856 -0.13712375  0.08361204  0.03010033] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.42056856 -0.11287625  0.41638796 -0.03010033] Diff

Epoch 1290 TotalParamVariation 0.00034699
[ 0.08361204 -0.37792642  0.14381271 -0.04347826] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[0.16638796 0.12792642 0.35618729 0.04347826] Diff

Epoch 1300 TotalParamVariation 0.00049331
[-0.04347826 -0.0367893   0.1638796   0.14381271] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.29347826 -0.2132107   0.3361204  -0.14381271] Diff

Epoch 1310 TotalParamVariation 0.00028261
[ 0.20401338 -0.08361204  0.22408027  0.07692308] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.04598662 -0.16638796  0.27591973 -0.07692308] Diff


Epoch 1830 TotalParamVariation 0.00024916
[ 0.24414716 -0.14381271  0.27090301 -0.15719064] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.00585284 -0.10618729  0.22909699  0.15719064] Diff

Epoch 1840 TotalParamVariation 0.00021321
[ 0.26421405  0.04347826  0.38461538 -0.00334448] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[-0.01421405 -0.29347826  0.11538462  0.00334448] Diff

Epoch 1850 TotalParamVariation 0.0001898
[ 0.33110368 -0.19063545  0.32441472 -0.06354515] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[-0.08110368 -0.05936455  0.17558528  0.06354515] Diff

Epoch 1860 TotalParamVariation 0.0002592
[ 0.10367893 -0.17056856  0.35117057  0.14381271] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.14632107 -0.07943144  0.14882943 -0.14381271] Diff

Epoch 1870 TotalParamVariation 0.00029264
[ 0.14381271 -0.090301    0.31772575 -0.13712375] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.10618729 -0.159699    0.18227425  0.13712375] Dif

...

Epoch 12980 TotalParamVariation 0.0001597
[ 0.09698997 -0.31772575  0.4916388  -0.090301  ] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[0.15301003 0.06772575 0.0083612  0.090301  ] Diff

Epoch 12990 TotalParamVariation 0.00020569
[ 0.41137124 -0.31103679  0.57859532 -0.11036789] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[-0.16137124  0.06103679 -0.07859532  0.11036789] Diff

Epoch 13000 TotalParamVariation 0.00013629
[ 0.33110368 -0.23076923  0.41137124 -0.08361204] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[-0.08110368 -0.01923077  0.08862876  0.08361204] Diff

Epoch 13010 TotalParamVariation 0.00020067
[ 0.22408027 -0.00334448  0.56521739  0.06354515] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.02591973 -0.24665552 -0.06521739 -0.06354515] Diff

Epoch 13020 TotalParamVariation 0.00018478
[ 0.17725753 -0.37123746  0.63210702  0.04347826] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.07274247  0.12123746 -0.13210702 -0.04347826] D

Epoch 13420 TotalParamVariation 0.00022993
[ 0.05685619 -0.27759197  0.47157191  0.21070234] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.19314381  0.02759197  0.02842809 -0.21070234] Diff

Epoch 13430 TotalParamVariation 0.00016806
[ 0.19732441 -0.28428094  0.63210702 -0.11705686] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.05267559  0.03428094 -0.13210702  0.11705686] Diff

Epoch 13440 TotalParamVariation 0.00030435
[ 0.2173913  -0.00334448  0.71906355  0.11036789] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.0326087  -0.24665552 -0.21906355 -0.11036789] Diff

Epoch 13450 TotalParamVariation 0.00018813
[ 0.33779264 -0.12374582  0.6187291   0.04347826] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[-0.08779264 -0.12625418 -0.1187291  -0.04347826] Diff

Epoch 13460 TotalParamVariation 9.532e-05
[ 0.22408027 -0.1638796   0.43812709 -0.01672241] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.02591973 -0.0861204   0.06187291  0.0167224

Epoch 13850 TotalParamVariation 0.00023161
[ 0.31103679 -0.07692308  0.70568562  0.02341137] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[-0.06103679 -0.17307692 -0.20568562 -0.02341137] Diff

Epoch 13860 TotalParamVariation 0.0001806
[ 0.33110368 -0.26421405  0.33779264 -0.10367893] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[-0.08110368  0.01421405  0.16220736  0.10367893] Diff

Epoch 13870 TotalParamVariation 0.00021572
[ 0.33779264 -0.39799331  0.56521739 -0.13043478] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[-0.08779264  0.14799331 -0.06521739  0.13043478] Diff

Epoch 13880 TotalParamVariation 0.00016472
[ 0.13043478 -0.37123746  0.57190635  0.01672241] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.11956522  0.12123746 -0.07190635 -0.01672241] Diff

Epoch 13890 TotalParamVariation 0.00023997
[ 0.31772575 -0.01672241  0.35117057  0.03010033] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[-0.06772575 -0.23327759  0.14882943 -0.0301003

Epoch 14260 TotalParamVariation 0.00011789
[ 0.30434783 -0.19063545  0.59866221  0.02341137] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[-0.05434783 -0.05936455 -0.09866221 -0.02341137] Diff

Epoch 14270 TotalParamVariation 0.0002291
[ 0.23076923 -0.07692308  0.24414716 -0.01003344] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.01923077 -0.17307692  0.25585284  0.01003344] Diff

Epoch 14280 TotalParamVariation 0.00015719
[ 0.17056856 -0.05016722  0.51170569 -0.02341137] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.07943144 -0.19983278 -0.01170569  0.02341137] Diff

Epoch 14290 TotalParamVariation 0.00028094
[ 0.05016722 -0.10367893  0.55183946 -0.1638796 ] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.19983278 -0.14632107 -0.05183946  0.1638796 ] Diff

Epoch 14300 TotalParamVariation 8.946e-05
[ 0.1638796  -0.25752508  0.45819398 -0.04347826] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[0.0861204  0.00752508 0.04180602 0.04347826] Di

Epoch 14680 TotalParamVariation 0.00020819
[ 0.23076923 -0.42474916  0.62541806  0.09698997] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.01923077  0.17474916 -0.12541806 -0.09698997] Diff

Epoch 14690 TotalParamVariation 0.00016472
[ 0.27759197 -0.04347826  0.51839465 -0.07692308] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[-0.02759197 -0.20652174 -0.01839465  0.07692308] Diff

Epoch 14700 TotalParamVariation 0.00016137
[ 0.19732441 -0.43143813  0.50501672  0.08361204] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.05267559  0.18143813 -0.00501672 -0.08361204] Diff

Epoch 14710 TotalParamVariation 0.0001898
[ 0.23745819 -0.25752508  0.23076923  0.090301  ] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.01254181  0.00752508  0.26923077 -0.090301  ] Diff

Epoch 14720 TotalParamVariation 0.00012542
[ 0.27759197 -0.36454849  0.51170569  0.09698997] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[-0.02759197  0.11454849 -0.01170569 -0.0969899

In [24]:
s_ising.train( max_epochs = 10000)

Epoch 100 TotalParamVariation 0.00048207
[ 0.0044475  -0.02263148  0.02340584 -0.01461645] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.2455525  -0.22736852  0.47659416  0.01461645] Diff

Epoch 200 TotalParamVariation 0.00045857
[ 0.01642473 -0.03371435  0.0466263  -0.01390356] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.23357527 -0.21628565  0.4533737   0.01390356] Diff

Epoch 300 TotalParamVariation 0.00043625
[ 0.02781353 -0.04424953  0.06867017 -0.01322543] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.22218647 -0.20575047  0.43132983  0.01322543] Diff

Epoch 400 TotalParamVariation 0.00041505
[ 0.03864053 -0.05426267  0.08958193 -0.01258037] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.21135947 -0.19573733  0.41041807  0.01258037] Diff

Epoch 500 TotalParamVariation 0.00039492
[ 0.04893175 -0.06377866  0.10940878 -0.01196676] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.20106825 -0.18622134  0.39059122  0.01196676] Diff



Epoch 5700 TotalParamVariation 3.664e-05
[ 0.2336963  -0.23483998  0.45907338 -0.0008883 ] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.0163037  -0.01516002  0.04092662  0.0008883 ] Diff

Epoch 5800 TotalParamVariation 3.51e-05
[ 0.23444907 -0.23553957  0.46065586 -0.00084497] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.01555093 -0.01446043  0.03934414  0.00084497] Diff

Epoch 5900 TotalParamVariation 3.363e-05
[ 0.23516683 -0.23620665  0.46217446 -0.00080375] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.01483317 -0.01379335  0.03782554  0.00080375] Diff

Epoch 6000 TotalParamVariation 3.222e-05
[ 0.23585122 -0.23684275  0.46363193 -0.00076454] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.01414878 -0.01315725  0.03636807  0.00076454] Diff

Epoch 6100 TotalParamVariation 3.087e-05
[ 0.23650383 -0.23744933  0.46503093 -0.00072724] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data
[ 0.01349617 -0.01255067  0.03496907  0.00072724] Diff

E